#Import

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

#Get API

In [ ]:
SERPAPI_KEY = '3e33047b5d45eba0a802a51a9deed3080ae17717eb542cd3bad23ff48126a1cb'

#Search Function

In [ ]:
def get_serp_results(query):
    search_url = f'https://serpapi.com/search?q={query}&api_key={SERPAPI_KEY}'
    response = requests.get(search_url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

def extract_email(text):
    text = BeautifulSoup(text, "html.parser").get_text()
    text = text.replace("[at]", "@").replace("[dot]", ".")

    email_regex = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    emails = re.findall(email_regex, text)

    return emails if emails else "N/A"

def find_all_links(soup):
    all_links = [a['href'] for a in soup.find_all('a', href=True)]
    return all_links if all_links else "N/A"

In [ ]:
def extract_company_info(query):
    results = get_serp_results(query)

    if results and 'organic_results' in results:
        company_info = []

        for result in results['organic_results']:
            company_name = result.get('title', 'N/A')
            company_url = result.get('link', 'N/A')

            try:

                response = requests.get(company_url, timeout=10)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')

                    page_emails = extract_email(soup.get_text())

                    all_links = find_all_links(soup)
                else:
                    page_emails = ["N/A"]
                    all_links = ["N/A"]
            except requests.exceptions.RequestException as e:
                print(f"Error fetching {company_url}: {e}")
                page_emails = ["N/A"]
                all_links = ["N/A"]

            company_info.append({
                'company_name': company_name,
                'company_url': company_url,
                'informal_emails': page_emails,
                'links': all_links
            })

            time.sleep(2)

        return company_info
    else:
        print("No results found.")
        return []

# Contoh Penggunaan
query = "Digital Marketing Agency"
company_data = extract_company_info(query)

# Result
for company in company_data:
    print("\nCompany Name:", company['company_name'])
    print("Company URL:", company['company_url'])
    print("Emails:", company['informal_emails'])
    print("Links:", company['links'][:5])  # Print 5 link pertama


Company Name: 10+ Digital Marketing Agencies in Chicago- 2025
Company URL: https://www.dmthriveagency.com/digital-marketing-agencies-in-chicago/
Emails: ['contact@dmthriveagency.com', 'dmthriveagency@gmail.com']
Links: ['#content', 'https://www.linkedin.com/company/dmthrive-agency', 'https://x.com/Dmthriveagency', 'https://www.instagram.com/dmthriveagency/', 'https://join.skype.com/invite/uWD7NsiFPO3d']

Company Name: Marcel Digital: Chicago Digital Marketing & Web Development ...
Company URL: https://www.marceldigital.com/
Emails: N/A
Links: ['#site-content', '/', '/services', '/services/search-engine-optimization', '/services/search-engine-optimization/technical-seo']

Company Name: #1 Digital Marketing Agency | Web Marketing Company
Company URL: https://coalitiontechnologies.com/digital-marketing
Emails: ['N/A']
Links: ['N/A']

Company Name: Thrive Internet Marketing Agency: Digital Marketing Agency
Company URL: https://thriveagency.com/
Emails: N/A
Links: ['https://thriveagency.co